In [2]:
import numpy as np
import torch

In [3]:
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [4]:
pt_path1 = '../results/sw/sw_CAE_h32_k5_ReZero_default_none_none/sw_CAE_h32_k5_ReZero_default_none_none_sigma_z_b=0.1_sigma_m=0.001_ens_num=32_infl=1.1_method=ETKF_assimilated.pt'
pt_path2 = '../results/sw/sw_AEflow_b4_k5_ReZero_default_none_none/sw_AEflow_b4_k5_ReZero_default_none_none_sigma_z_b=0.1_sigma_m=0.001_ens_num=32_infl=1.05_method=ETKF_assimilated.pt'
pt_path3 = '../results/sw/sw_SINRv11_default_NeuralODE_no_exp_none_none/sw_SINRv11_default_NeuralODE_no_exp_none_none_sigma_z_b=0.03_sigma_m=0.001_ens_num=32_infl=1.02_method=DEnKF_assimilated.pt'

In [5]:
ass_data1 = torch.load(pt_path1)
ass_data2 = torch.load(pt_path2)
ass_data3 = torch.load(pt_path3)

ass_data1['xx_a'].shape

torch.Size([199, 1024])

In [6]:
import sys

sys.path.append('../')
from datasets import MetaData

dataset_ts_path = '/home/lizhuoyuan/datasets/shallow_water/cached_meta_ts.pt'
dataset_ts:MetaData = torch.load(dataset_ts_path)
dataset_ts.trajs[0].shape

torch.Size([240, 128, 64, 2])

In [7]:
import os
import logging
from datasets import load_dataset

logger = logging.getLogger()

In [8]:
from components import EncoderDecoder, LatentDynamics, UncertaintyEst
from components import get_encoder_decoder, get_latent_dynamics, get_uncertainty_est

In [9]:
import hydra
from hydra import compose, initialize
from hydra.core.config_store import ConfigStore
from omegaconf import OmegaConf

from configs.assimilate.assimilate_conf_schema import AssimilateConfig
from configs.finetune.finetune_conf_schema import FineTuneConfig
from configs.conf_schema import EDConfig, LDConfig, UEConfig, DatasetConfig
from metrics import get_metrics
from common import create_logger, set_seed, transform_state_dict


cs = ConfigStore.instance()
cs.store(name="assimilate_schema", node=AssimilateConfig)
cs.store(name="finetune_schema", node=FineTuneConfig)
cs.store(group="encoder_decoder", name="encoder_decoder_schema", node=EDConfig)
cs.store(group="latent_dynamics", name="latent_dynamics_schema", node=LDConfig)
cs.store(group="uncertainty_est", name="uncertainty_est_schema", node=UEConfig)
cs.store(name="dataset_schema", group='dataset', node=DatasetConfig)

with initialize(config_path='../configs/finetune', version_base='1.2'):
    cfg1 = compose(config_name='config', overrides=[
        "encoder_decoder=ablation/cae/h32_k5",
        'latent_dynamics=default_rezero',
    ])
    cfg2 = compose(config_name='config', overrides=[
        "encoder_decoder=ablation/aeflow/b4_k5",
        'latent_dynamics=default_rezero',
    ])
    cfg3 = compose(config_name='config', overrides=[
        "encoder_decoder=default_sinr_v11",
        'latent_dynamics=ablation/neuralode/no_exp',
    ])


In [10]:
def conf_prepare(cfg: FineTuneConfig):

    cfg.encoder_decoder.arch_params.state_channels = cfg.dataset.snapshot_shape[-1]
    cfg.encoder_decoder.arch_params.state_size = cfg.dataset.snapshot_shape[:-1]
    cfg.latent_dynamics.latent_dim = cfg.encoder_decoder.latent_dim
    
    cfg.ckpt_path=os.path.join('..', cfg.ckpt_path)

conf_prepare(cfg1)
conf_prepare(cfg2)
conf_prepare(cfg3)

print(OmegaConf.to_yaml(cfg1))

name: ${dataset.name}_${encoder_decoder.name}_${latent_dynamics.name}
phase: finetune
seed: 2357
num_gpus: 0
master_port: 23571
ckpt_path: ../../ckpts/finetune/sw_CAE_h32_k5_ReZero_default.pt
nepochs: 1000
bs: ${encoder_decoder.training_params.bs}
eval_freq: 10
mix_precision: false
pretrain_name: ${dataset.name}_${encoder_decoder.name}
pretrain_ckpt_path: ../ckpts/pretrain/${pretrain_name}.pt
dataset:
  name: sw
  root_path: /home/lizhuoyuan/datasets/shallow_water
  snapshot_shape:
  - 128
  - 64
  - 2
  read_cache: true
  normalize: true
  normalize_mean: null
  normalize_std: null
  window_width: 10
encoder_decoder:
  model_name: CAE
  cfg_name: h32_k5
  name: ${encoder_decoder.model_name}_${encoder_decoder.cfg_name}
  latent_dim: 1024
  need_train: true
  need_cache: false
  arch_params:
    hidden_channels: 32
    latent_channels: 8
    kernel_size: 5
    padding_type:
    - circular
    - replicate
    nresblocks: null
    state_size:
    - 128
    - 64
    state_channels: 2
    c

In [11]:
dataset_class = load_dataset(logger, cfg=cfg1.dataset)
dataset_tr = dataset_class.get_metadata('tr')
dataset_va = dataset_class.get_metadata('va')
dataset_ts = dataset_class.get_metadata('ts')

In [12]:
def get_model(cfg: FineTuneConfig):
    if cfg.encoder_decoder.need_cache:
        loss_fn_inner_loop = get_metrics(name=cfg.encoder_decoder.arch_params.inner_loop_loss_fn,
                                         phi_theta=dataset_va.coords['coord_latlon'])
    else:
        loss_fn_inner_loop = None

    model: EncoderDecoder = get_encoder_decoder(logger,
                                                name=cfg.encoder_decoder.model_name,
                                                loss_fn_inner_loop=loss_fn_inner_loop,
                                                **cfg.encoder_decoder.arch_params)
    ckpt = torch.load(cfg.ckpt_path, map_location='cpu')
    model.load_state_dict(transform_state_dict(ckpt['ed']))

    return model

In [13]:
model1 = get_model(cfg1)
model2 = get_model(cfg2)
model3 = get_model(cfg3)

In [14]:
dataset_va.coords['coord_latlon'].shape

torch.Size([128, 64, 2])

In [15]:
ass_data3['xx_a'].shape

torch.Size([199, 400])

In [16]:
xx_a1 = model1(ass_data1['xx_a'], operation='decode')
xx_a2 = model2(ass_data2['xx_a'], operation='decode')
xx_a3 = model3(ass_data3['xx_a'], operation='decode', coord_latlon=dataset_va.coords['coord_latlon'][None, ...])

In [17]:
xx_a1 = xx_a1.detach().cpu().numpy()
xx_a2 = xx_a2.detach().cpu().numpy()
xx_a3 = xx_a3.detach().cpu().numpy()
xx_t = dataset_ts.trajs[0].detach().cpu().numpy()

In [26]:
print(f'{xx_a1.shape=},\n{xx_a2.shape=},\n{xx_a3.shape=},\n{xx_t.shape=}')

xx_a1.shape=(199, 128, 64, 2),
xx_a2.shape=(199, 128, 64, 2),
xx_a3.shape=(199, 128, 64, 2),
xx_t.shape=(240, 128, 64, 2)


In [27]:
xx_t = xx_t[1:200]

In [18]:
from typing import *
import matplotlib as mpl
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

vmin=-1, vmax=1
vmin=-3, vmax=3


/tmp/ipykernel_12422/2637377777.py:122: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [53]:
fig = plt.figure(figsize=(12, 8))

# ------------------------------ height ------------------------------

imgrid = ImageGrid(fig, 211, nrows_ncols=(4, 6), axes_pad=.05,
                   #    share_all=True,
                   cbar_pad='3%',
                   cbar_size="3%",
                   cbar_location="right", cbar_mode="single")  # [0, 48, 96, 144, 192]

# axs: Iterable[Iterable[Axes]]
idz = 0

vmin = -1
vmax = 1


cmap = mpl.cm.coolwarm
bounds = list(np.linspace(vmin, vmax, 61))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')

print(f'{vmin=}, {vmax=}')

xlabels = ['Reference', 'CAE+ReZero', 'AEflow+ReZero', 'LAINR (ours)']
steps = [1] + list(range(48, 48 * 5, 48))

for idx, xx in enumerate([xx_t, xx_a1, xx_a2, xx_a3]):
    for idy, step in enumerate(steps):
        im = imgrid[idz].imshow(xx[step - 1][..., 0].T,
                                origin='lower',
                                cmap=cmap, norm=norm)
        if idy == 0:
            imgrid[idz].set_ylabel(xlabels[idx], fontsize=8)
        idz += 1
    if idx > 0:
        imgrid[idz].imshow(xx[-1][..., 0].T - xx_t[-1][..., 0].T,
                           origin='lower',
                           cmap=cmap, norm=norm)
    else:
        imgrid[idz].plot([0, 127], [0, 63], color='r', linewidth=2)
    idz += 1

imgrid[18 + 0].set_xlabel(f'$\\Delta t={steps[0]}$ h', usetex=True)
imgrid[18 + 1].set_xlabel(f'$\\Delta t={steps[1]}$ h', usetex=True)
imgrid[18 + 2].set_xlabel(f'$\\Delta t={steps[2]}$ h', usetex=True)
imgrid[18 + 3].set_xlabel(f'$\\Delta t={steps[3]}$ h', usetex=True)
imgrid[18 + 4].set_xlabel(f'$\\Delta t={steps[4]}$ h', usetex=True)
imgrid[18 + 5].set_xlabel(f'relative errors')

for ax in imgrid:
    ax.set_xticks([])
    ax.set_yticks([])

cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=imgrid.cbar_axes[0],
    cax=imgrid.cbar_axes[0],
    orientation='vertical', label='height (normalized)')

cbar.set_ticks(list(range(-1, 2)), minor=False)

# ------------------------------ vorticity ------------------------------

imgrid = ImageGrid(fig, 212, nrows_ncols=(4, 6), axes_pad=.05,
                   #    share_all=True,
                   cbar_pad='3%',
                   cbar_size="3%",
                   cbar_location="right", cbar_mode="single")  # [0, 48, 96, 144, 192]

# axs: Iterable[Iterable[Axes]]
idz = 0

vmin = xx_t[..., 1].min()
vmax = xx_t[..., 1].max()

vmin = -3
vmax = 3

cmap = mpl.cm.coolwarm
bounds = list(np.linspace(-3, 3, 61))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')

print(f'{vmin=}, {vmax=}')

xlabels = ['Reference', 'CAE+ReZero', 'AEflow+ReZero', 'LAINR (ours)']
steps = [1] + list(range(48, 48 * 5, 48))

for idx, xx in enumerate([xx_t, xx_a1, xx_a2, xx_a3]):
    for idy, step in enumerate(steps):
        im = imgrid[idz].imshow(xx[step - 1][..., 1].T,
                                origin='lower',
                                cmap=cmap, norm=norm)
        if idy == 0:
            imgrid[idz].set_ylabel(xlabels[idx], fontsize=8)
        idz += 1
    if idx > 0:
        imgrid[idz].imshow(xx[-1][..., 1].T - xx_t[-1][..., 1].T,
                           origin='lower',
                           cmap=cmap, norm=norm)
    else:
        imgrid[idz].plot([0, 127], [0, 63], color='r', linewidth=2)
    idz += 1

imgrid[18 + 0].set_xlabel(f'$\\Delta t={steps[0]}$ h', usetex=True)
imgrid[18 + 1].set_xlabel(f'$\\Delta t={steps[1]}$ h', usetex=True)
imgrid[18 + 2].set_xlabel(f'$\\Delta t={steps[2]}$ h', usetex=True)
imgrid[18 + 3].set_xlabel(f'$\\Delta t={steps[3]}$ h', usetex=True)
imgrid[18 + 4].set_xlabel(f'$\\Delta t={steps[4]}$ h', usetex=True)
imgrid[18 + 5].set_xlabel(f'relative errors')

for ax in imgrid:
    ax.set_xticks([])
    ax.set_yticks([])

cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=imgrid.cbar_axes[0],
    cax=imgrid.cbar_axes[0],
    orientation='vertical', label='vorticity (normalized)')

cbar.set_ticks(list(range(-3, 4)), minor=False)
plt.tight_layout()
plt.savefig('sw-stamppic.pdf')
plt.show()

In [62]:
import pandas as pd
import seaborn as sns

In [63]:
rmse_fn = get_metrics('weighted_root', phi_theta=dataset_va.coords['coord_latlon'])
# xx_a1 = torch.from_numpy(xx_a1)
# xx_a2 = torch.from_numpy(xx_a2)
# xx_a3 = torch.from_numpy(xx_a3)
# xx_t = torch.from_numpy(xx_t)

models = ['CAE+ReZero', 'AEflow+ReZero', 'LAINR (ours)']
rows_list=[]

for idx, xx in enumerate([xx_a1, xx_a2, xx_a3]):
    for step, (x, x_t) in enumerate(zip(xx.split(1), xx_t.split(1))):
        rmse = rmse_fn(x, x_t, feature_sep=True)
        rows_list.append({'model': models[idx], 'step': step, 'feature': 'height', 'rmse': rmse[0].item()})
        rows_list.append({'model': models[idx], 'step': step, 'feature': 'vorticity', 'rmse': rmse[1].item()})

df = pd.DataFrame(rows_list)
        
df
# rmse1 = rmse_fn(torch.from_numpy(xx_a1), torch.from_numpy(xx_t), feature_sep=True)
# rmse2 = rmse_fn(torch.from_numpy(xx_a2), torch.from_numpy(xx_t), feature_sep=True)
# rmse3 = rmse_fn(torch.from_numpy(xx_a3), torch.from_numpy(xx_t), feature_sep=True)

,model,step,feature,rmse
0,CAE+ReZero,0,height,0.053694
1,CAE+ReZero,0,vorticity,0.156855
2,CAE+ReZero,1,height,0.053700
3,CAE+ReZero,1,vorticity,0.156192
4,CAE+ReZero,2,height,0.053260
...,...,...,...,...
1189,LAINR (ours),196,vorticity,0.104029
1190,LAINR (ours),197,height,0.045430
1191,LAINR (ours),197,vorticity,0.104506
1192,LAINR (ours),198,height,0.045211


In [72]:
ax=sns.lineplot(data=df, x='step', y='rmse', hue='model', style='feature',palette='deep')
ax.set_yscale('log')
ax.set_xlabel('assimilation cycles')
ax.set_ylabel('RMSE')
plt.legend(ncol=2)
plt.tight_layout()
plt.savefig('sw-rmse.pdf')